# graph analysis

In [1]:
import config
import geopandas as gpd
import json
import networkx as nx
import numpy as np
import os
import osmnx as ox
import pandas as pd
import time

ox.config(use_cache=True,
          log_file=True,
          log_console=True,
          log_filename='calculate-cities',
          cache_folder="F:/ITC/proposal/test")

print(ox.__version__)
print(nx.__version__)

0.10
2.2


In [315]:
graphml_folder ='folder'# place graphml files
places_folder ='folder' # place shapefiles
folder ='name' #the name of created folder for each country

creatr a datafram based on the attribute of shapefile

In [317]:
places = []
for state_folder in folder:
    for city_file in os.listdir('{}/{}'.format(graphml_folder, folder)):

        data = {}
        data['state_folder'] = folder
        data['iso3'] = folder.split('_')[0]
        data['name'] = folder.split('_')[1]
        data['city_file'] = city_file
        data['fuacode_si'] = city_file.split('_')[0]
        data['fuaname_en'] = city_file.strip('_{}'.format(data['fuacode_si'])).replace('.graphml', '').replace('_', ' ')
        
        places.append(data)

df = pd.DataFrame(places)
print(df)

            city_file fuacode_si   fuaname_en iso3    name state_folder
0     SE001_Stockholm      SE001     tockholm  SWE  Sweden   SWE_Sweden
1    SE002_Gothenburg      SE002   Gothenburg  SWE  Sweden   SWE_Sweden
2         SE003_Malmo      SE003        Malmo  SWE  Sweden   SWE_Sweden
3     SE004_Jonkoping      SE004    Jonkoping  SWE  Sweden   SWE_Sweden
4          SE005_Umea      SE005         Umea  SWE  Sweden   SWE_Sweden
5       SE006_Uppsala      SE006      Uppsala  SWE  Sweden   SWE_Sweden
6     SE007_Linkoping      SE007    Linkoping  SWE  Sweden   SWE_Sweden
7        SE008_Orebro      SE008       Orebro  SWE  Sweden   SWE_Sweden
8      SE501_Vasteras      SE501     Vasteras  SWE  Sweden   SWE_Sweden
9    SE502_Norrkoping      SE502   Norrkoping  SWE  Sweden   SWE_Sweden
10  SE503_Helsingborg      SE503  Helsingborg  SWE  Sweden   SWE_Sweden
11        SE505_Boras      SE505        Boras  SWE  Sweden   SWE_Sweden


In [318]:
# load each state shapefile and get the geoid and aland for each city row
gdf = gpd.GeoDataFrame()
for x in df['name'].unique():
    path = '{}/{}'.format(places_folder, x)
    gdf = gdf.append(gpd.read_file(path)[['fuacode_si', 'area']])

# merge aland values into dataframe, on geoid
gdf = gdf.rename(columns=str.lower)
df = pd.merge(df, gdf, how='left', on='fuacode_si')

In [319]:
folder = '{}/{}/{}'.format(graphml_folder, df['folder'],df['city_file'])

df['graph'] = df['city_file'].apply(lambda x: "{}.{}".format(x, 'graphml'))
filename=df['graph']

In [320]:
print(df)

            city_file fuacode_si   fuaname_en iso3    name state_folder  \
0     SE001_Stockholm      SE001     tockholm  SWE  Sweden   SWE_Sweden   
1    SE002_Gothenburg      SE002   Gothenburg  SWE  Sweden   SWE_Sweden   
2         SE003_Malmo      SE003        Malmo  SWE  Sweden   SWE_Sweden   
3     SE004_Jonkoping      SE004    Jonkoping  SWE  Sweden   SWE_Sweden   
4          SE005_Umea      SE005         Umea  SWE  Sweden   SWE_Sweden   
5       SE006_Uppsala      SE006      Uppsala  SWE  Sweden   SWE_Sweden   
6     SE007_Linkoping      SE007    Linkoping  SWE  Sweden   SWE_Sweden   
7        SE008_Orebro      SE008       Orebro  SWE  Sweden   SWE_Sweden   
8      SE501_Vasteras      SE501     Vasteras  SWE  Sweden   SWE_Sweden   
9    SE502_Norrkoping      SE502   Norrkoping  SWE  Sweden   SWE_Sweden   
10  SE503_Helsingborg      SE503  Helsingborg  SWE  Sweden   SWE_Sweden   
11        SE505_Boras      SE505        Boras  SWE  Sweden   SWE_Sweden   

            area        

In [321]:
def load_graph_get_stats(row):
    
    try:
        start_time = time.time()
        folder = '{}/{}/{}'.format(graphml_folder, row['folder'],row['city_file'])
        filename=row['graph']
        #filename=filename.append('{}.graphml')
        G = ox.load_graphml(filename=filename, folder=folder)
        
        
        stats = ox.basic_stats(G, area=row['area'])
        edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
        
        #d-number
        count=edges['highway'].value_counts()
        count_frame=count.to_frame()
        count_main=[]
        for i in count.index:
            if type(i) is str:
                x = count_frame.at[i, 'highway']
                count_main.append(x)
        N=count_main
        T_N=sum(N)
        D=[]
        for i in np.arange(0,len(N),1):
            E =N[i]*(N[i]-1)/(T_N*(T_N-1))
            D.append(E)
        summ_D= sum(D)
        diversity_D_number=1-summ_D
        stats.update( {'diversity_D_number' : diversity_D_number} )
    
    
       #h-number
        d=[]
        for i in np.arange(0,len(N),1):
            e =-(N[i]/T_N)*(np.log(N[i]/T_N))
            d.append(e)
        summ_H= sum(d)
        diversity_H_number=summ_H
        stats.update( {'diversity_H_number' : diversity_H_number} )
    
    
    
        #h-length
        length=[]
        for i in count.index:
            if type(i) is str:
                x = edges.loc[edges['highway'] ==i, 'length']
                x =x .to_frame()
                x  = x['length'].sum()
                length.append(x)    
        total=sum(length)
        H=[]
        for i in length:        
            R=-(i/total)*(np.log(i/total))
            H.append(R)
        diversity_H_len= sum(H)
        stats.update( {'diversity_H_len' : diversity_H_len} )
    
    
    
        #d-length
        h=[]
        for i in length:        
            W =i*(i-1)/(total*(total-1))
            h.append(W)
        diversity_D_len=1- sum(h)
        stats.update( {'diversity_D_len' : diversity_D_len} )
    
  
    
    
        #entropy
        Gu = ox.add_edge_bearings(G)
        bearings = pd.Series([data['bearing'] for u, v, k,  data in Gu.edges(keys=True, data=True)])
        n = 36
        n = n * 2
        bins = np.arange(n + 1) * 360 / n
        count, _ = np.histogram(bearings, bins=bins)
        count = np.roll(count, 1)
        count=count[::2] + count[1::2]
        x=count.tolist()
        total=sum(x)
        H=[]
        for i in np.arange(0,len(x),1):
            E =-(x[i]/total)*(np.log(x[i]/total))
            H.append(E)
        E1= sum(H)
        Emax=np.log(36)
        Eg=1.386
        E=1-((E1-Eg)/(Emax-Eg))*((E1-Eg)/(Emax-Eg))    
        stats.update( {'entropy' : E} )
        
        
        # unpack k-counts and k-proportion dicts into individiual keys:values
        for k, count in stats['streets_per_node_counts'].items():
            stats['int_{}_streets_count'.format(k)] = count
        for k, proportion in stats['streets_per_node_proportion'].items():
            stats['int_{}_streets_prop'.format(k)] = proportion
            
        # calculate/drop the extended stats that have values per node
        extended_stats = ox.extended_stats(G)
        se = pd.Series(extended_stats)
        se = se.drop(['avg_neighbor_degree', 'avg_weighted_neighbor_degree', 'clustering_coefficient',
                      'clustering_coefficient_weighted', 'degree_centrality', 'pagerank'])
        extended_stats_clean = se.to_dict()
        
        for key in extended_stats_clean:
            stats[key] = extended_stats_clean[key]
        
        stats['area_km'] = row['area'] / 1e6        
        stats['city'] = row['fuaname_en']
        stats['state'] = row['name']
        stats['geoid'] = row['fuacode_si']
        stats['area'] = row['area']
        stats['time'] = time.time()-start_time
        return pd.Series(stats)

    except Exception as e:
        print('{}, {} failed: {}'.format(row['city'], row['state'], e))
        return pd.Series()

In [322]:
#sample = list(range(0, len(df), int(len(df)/100)))
#stats = df.iloc[sample].apply(load_graph_get_stats, axis=1)
stats_temp = df.apply(load_graph_get_stats, axis=1)
stats_temp.shape

(12, 58)

In [ ]:
stats_temp['time'].sum()
         

In [314]:
stats = stats_temp.copy()

In [296]:
# stuff to drop
cols_to_drop = ['area', 'time', 'streets_per_node_counts', 'streets_per_node_proportion', 
                'pagerank_max_node', 'pagerank_min_node', 'clean_intersection_count',
                'clean_intersection_density_km']

In [297]:
cols_to_rename = {}
for col in stats.columns:
    if 'int_' in col:
        n = col.split('_')[1]
        if n not in ['1', '3', '4']:
            cols_to_drop.append(col)
        else:
            suffix = 'count' if 'count' in col else 'proportion'
            cols_to_rename[col] = 'intersect_{}way_{}'.format(n, suffix)
            
stats = stats.drop(cols_to_drop, axis=1)

In [298]:
# rename these to friendlier names
cols_to_rename['clustering_coefficient_avg'] = 'cluster_coeff_avg'
cols_to_rename['clustering_coefficient_weighted_avg'] = 'cluster_coeff_weighted_avg'
cols_to_rename['intersection_density_km'] = 'intersect_density_km'
cols_to_rename['intersect_1way_count'] = 'dead_end_count'
cols_to_rename['intersect_1way_proportion'] = 'dead_end_proportion'
cols_to_rename['m'] = 'edge_count'
cols_to_rename['n'] = 'node_count'
stats = stats.rename(columns=cols_to_rename)
stats = stats.rename(columns=cols_to_rename)

In [299]:
# drop anything lacking a GEOID
stats = stats.dropna(subset=['geoid'])

In [300]:
# make these integers
cols_int = ['intersection_count', 'edge_length_total', 'edge_count', 'node_count', 'street_segments_count']
stats[cols_int] = stats[cols_int].astype(np.int64)

In [301]:
# make city, state, geoid at left of df
cols = stats.columns.tolist()
cols.insert(0, cols.pop(cols.index('city')))
cols.insert(1, cols.pop(cols.index('state')))
cols.insert(2, cols.pop(cols.index('geoid')))
stats = stats.reindex(columns=cols)

In [302]:
stats.shape

(81, 38)

In [303]:
stats.columns

Index(['city', 'state', 'geoid', 'area_km', 'avg_neighbor_degree_avg',
       'avg_weighted_neighbor_degree_avg', 'circuity_avg', 'cluster_coeff_avg',
       'cluster_coeff_weighted_avg', 'degree_centrality_avg',
       'diversity_D_len', 'diversity_D_number', 'diversity_H_len',
       'diversity_H_number', 'edge_density_km', 'edge_length_avg',
       'edge_length_total', 'entropy', 'dead_end_count', 'dead_end_proportion',
       'intersect_3way_count', 'intersect_3way_proportion',
       'intersect_4way_count', 'intersect_4way_proportion',
       'intersection_count', 'intersect_density_km', 'k_avg', 'edge_count',
       'node_count', 'node_density_km', 'pagerank_max', 'pagerank_min',
       'self_loop_proportion', 'street_density_km', 'street_length_avg',
       'street_length_total', 'street_segments_count', 'streets_per_node_avg'],
      dtype='object')

In [304]:
stats.head()

,city,state,geoid,area_km,avg_neighbor_degree_avg,avg_weighted_neighbor_degree_avg,circuity_avg,cluster_coeff_avg,cluster_coeff_weighted_avg,degree_centrality_avg,...,node_count,node_density_km,pagerank_max,pagerank_min,self_loop_proportion,street_density_km,street_length_avg,street_length_total,street_segments_count,streets_per_node_avg
0,Madrid,Spain,ES001,1473.990,2.076407,0.054911,1.055438,0.063325,0.000554,0.000061,...,65784,44.629882,0.000080,0.000002,0.002737,6969.374859,104.216094,1.027279e+07,98572,2.998617
1,Barcelona,Spain,ES002,714.982,2.075209,0.053353,1.065403,0.065556,0.000538,0.000089,...,44748,62.586191,0.000130,0.000003,0.001754,9306.104464,96.655924,6.653697e+06,68839,3.082998
2,Valencia,Spain,ES003,416.358,2.114225,0.055468,1.052252,0.068038,0.000574,0.000173,...,23309,55.983072,0.000203,0.000006,0.001129,8054.338202,95.288499,3.353488e+06,35193,3.030031
3,eville,Spain,ES004,673.770,2.215516,0.054370,1.054748,0.060268,0.000431,0.000185,...,22881,33.959660,0.000251,0.000007,0.002227,4807.338656,95.319166,3.239041e+06,33981,2.971592
4,aragossa,Spain,ES005,1006.160,1.917614,0.048696,1.073749,0.097122,0.000915,0.000374,...,9904,9.843365,0.000512,0.000015,0.002941,1785.297678,122.105575,1.796295e+06,14711,2.986066


In [305]:
if not os.path.exists(stats_folder):
    os.makedirs(stats_folder)
output_path = '{}/{}/cities-stats.csv'.format(graphml_folder, stats_folder)

stats.to_csv(output_path, encoding='utf-8', index=False)